Without NEL for now, just pattern matching, and linguistic heuristics to remove unpromising sentences from the pipeline



[ 1 ] Sentence Level check Filter before NER/NEL/neg Basic Tagger + Tokenizer
    => Len, SDP-len, Dep_complexity. TRIGGER WORDS! (cause, inhibit, therefore)
 
[ 2 ] Remaining sentences (with doc context needed for NER context??) =>


[ 3 ] Use construct_pattern and GNBR dep paths to look at hits in different datasets

[ 4 ] Implement something akin to an AllenAI Spike workflow
        1) Search syntax pattern
        2) System shows related patterns (embedding space)
        3) See variate of surface forms --> batch annotate?

In [1]:
%load_ext autoreload
%autoreload 2

import spacy
from negspacy.negation import Negex #https://github.com/jenojp/negspacy
from spacy.pipeline import EntityRuler
from extractacy.extract import ValueExtractor

In [48]:
from scify.nlp import *
from scify.consts import labels
from scify.V import V


In [5]:
nlp = load_sci_pipe()
negex = Negex(nlp, language="en_clinical_sensitive")
nlp.add_pipe(negex, last=True) #works on an per entity basis. If causal phrases become entities?

In [6]:
from textacy import Corpus

In [7]:
GN = Corpus.load(nlp, "../data/experiments/GNBR_chem-gene_annotated_500")

In [57]:
from scify.utils.datasets import GNBR

In [58]:
data, distributions = GNBR.get_data_and_distributions()

['A+', 'A+.ind', 'A-', 'A-.ind', 'B', 'B.ind', 'E+', 'E+.ind', 'E-', 'E-.ind', 'E', 'E.ind', 'N', 'N.ind', 'O', 'O.ind', 'K', 'K.ind', 'Z', 'Z.ind'] headers 
                            chemical-gene
                    (A+) agonism, activation
                    (A-) antagonism, blocking
                    (B) binding, ligand (esp. receptors)
                    (E+) increases expression/production
                    (E-) decreases expression/production
                    (E) affects expression/production (neutral)
                    (N) inhibits
                            


In [59]:
GN[:6]

[Similarly 18F-FDG-PET/CT had higher sensitivity and specificity in hormone_receptor -LRB- + -RRB- and -LRB- - -RRB- groups .,
 The purpose of the present study was to investigate the protective effect of the -1 receptor -LRB- Sig-1R -RRB- agonist -LRB- + -RRB- - pentazocin -LRB- PTZ -RRB- on pressure-induced apoptosis and death of human trabecular meshwork cells -LRB- hTMCs -RRB- .,
 It decreased by 51 % cholinesterase inhibition in the brain caused by i.p. injection of 2 mg of parathion/kg body weight but not that of an equitoxic dose -LRB- 0.5 mg/kg -RRB- of its active metabolite , paraoxon -LRB- 0,0-diethyl_0-4-nitrophenyl_phosphate -RRB- .,
 Further analysis of the 01CM53122 genome showed that this virus represents a diverse set of mosaic genomes from CRF22_01A1 , including a 446-nt segment of 01CM53122 in the env region , but unlike other CRF22 strains , clustered with CRF01_AE rather than the A1 sequence , suggesting that the 01CM53122 strain is a recombinant of CRF22_01A1 and C

In [35]:
text = "Alzheimer's Disease causes Dementia in the older population"

In [28]:
[* doc.noun_chunks]

[Alzheimer's Disease, Dementia, the older population]

In [36]:
for token in doc:
    
            # to test for only verb forms 'is' and 'was' use token.head.lower_ in ['is', 'was']
    print([t.text for t in token.subtree])

["Alzheimer's Disease"]
["Alzheimer's Disease", 'causes', 'Dementia', ',', 'in', 'the', 'older population']
['Dementia', ',']
[',']
['in', 'the', 'older population']
['the']
['the', 'older population']


In [39]:
doc =nlp(text)

In [ ]:
#Alzheimer -> Dementia, meta: {+in the older population}

In [42]:
V.hierplane([*nlp("Alzheimer's Disease causes Dementia, in the older population").sents][0])

heelo


In [41]:
V.visualise_doc(doc, compact=True)

In [38]:
V.visualise_doc(doc, compact=True)

In [ ]:
#GNBR_preprocessing.GNBR_SUPPORT_LEGEND

In [ ]:
#Dependency paths with support
#{k for k in distributions if }

## Stats:

* ) What's the average lengths for the paths from least to most supported?
* ) Can we group them by edit_distance(2) and see if there's lots in common?
* ) strongest_paths precision on PubMed (chem->gene sentences)??
* ) How many hits? How much edit distance? Lemmatize? Make vars**??
* ) strongest support vs dep_length plot?
* ) max and averages

In [86]:
dep_paths = [k for k in distributions]
len(dep_paths)

801782

In [37]:
len([k for k in distributions if sum(distributions[k].values()) > 3])

355902

In [51]:
strong = [k for k in distributions if max(distributions[k].values()) > 2000]

In [52]:
def num_arcs(dep_path:str)->int:
    return len(dep_path.split(" "))


In [54]:
average = sum([num_arcs(dep) for dep in strong])/len(strong)
average

2.0294117647058822

In [88]:
deps = pd.DataFrame({"dep_path": dep_paths, "dep_arcs": [num_arcs(dep) for dep in dep_paths]})

In [90]:
alt.Chart(deps[:4000]).mark_bar().encode(x="dep_arcs:O", )

alt.Chart(...)

In [91]:
len([k for k in distributions if max(distributions[k].values()) > 4000])

25

In [85]:
import altair as alt
from vega_datasets import data
#source = data.wheat()

alt.Chart(source).mark_bar().encode(
    x='year:O',
    y="wheat:Q",
    # The highlight will be set on the result of a conditional statement
)

alt.Chart(...)

In [60]:
import pandas as pd
data = pd.DataFrame({'a': list('CCCDDDEEE'),
                     'b': [2, 7, 4, 1, 2, 6, 8, 4, 7]})

In [ ]:
dep_info = pd.DataFrame({"dep_str":  })

In [48]:
import seaborn as sns
sns.set(style="darkgrid")
titanic = sns.load_dataset("titanic")
ax = sns.countplot(x="class", data=titanic)

from textacy import Corpus
GN = Corpus.load("../data/experiments/GNBR_chem-gene_annotated_500")
%load_ext autoreload
%autoreload 2

import spacy
from negspacy.negation import Negex #https://github.com/jenojp/negspacy
from spacy.pipeline import EntityRuler
from extractacy.extract import ValueExtractor
from scify.nlp import *
from scify.consts import labels

nlp = load_sci_pipe()
from scify.nlp import *
from scify.consts import labels

nlp = load_sci_pipe()
%load_ext autoreload
%autoreload 2

import spacy
from negspacy.negation import Negex #https://github.com/jenojp/negspacy
from spacy.pipeline import EntityRuler
from extractacy.extract import ValueExtractor
from scify.nlp import *
from scify.consts import labels

nlp = load_sci_pipe()
from scify.nlp import *
from scify.consts import labels

nlp = load_sci_pipe()

negex = Negex(nlp, language="en_clinical_sensitive")
nlp.add_pipe(negex, last=True) #works on an per entity basis. If causal phrases become entities?
from textacy import Corpu

In [ ]:
#kinases|compound|start_entity participate|nsubj|kinases participate|nmod|end_entity	
#2.000000	0	0.000000	0	0.000000	0	0.000000	0	0.000000	0	0.000000	0	0.000000	0	2.000000	0	0.000000	0	0.000000	0

In [22]:
data[:2]

[{'pmid': '25640386',
  'sent': 'Similarly 18F-FDG-PET/CT had higher sensitivity and specificity in hormone_receptor -LRB- + -RRB- and -LRB- - -RRB- groups .',
  'ent1': '-',
  'ent1_offset': '1624,1625',
  'ent2': 'hormone_receptor',
  'ent2_offset': '1598,1614',
  'ent1_raw': '-',
  'ent2_raw': 'hormone receptor',
  'ent1_canonical': 'null',
  'ent2_canonical': '3164',
  'ent1_type': 'Chemical',
  'ent2_type': 'Gene',
  'dep': 'sensitivity|appos|START_ENTITY sensitivity|nmod|END_ENTITY'},
 {'pmid': '28560459',
  'sent': 'The purpose of the present study was to investigate the protective effect of the -1 receptor -LRB- Sig-1R -RRB- agonist -LRB- + -RRB- - pentazocin -LRB- PTZ -RRB- on pressure-induced apoptosis and death of human trabecular meshwork cells -LRB- hTMCs -RRB- .',
  'ent1': '+',
  'ent1_offset': '244,245',
  'ent2': 'Sig-1R',
  'ent2_offset': '227,233',
  'ent1_raw': '+',
  'ent2_raw': 'Sig-1R',
  'ent1_canonical': 'null',
  'ent2_canonical': '10280',
  'ent1_type': 'Chem

In [ ]:
#add_matches
#
#FOCUS on ONE DATASET WITH ONE PIPELINE ... TRAIN THE PATTERN MATCHING and EXTRACTION part, not the UTIL WRITING PART!!!!

#def SDP(doc:Doc, a:Union[Span, Token], b: Union[Span, Token])->List[Union[Span,Token]]:

get_sdp_path(doc, 0, 7)
#=> have|nsubj|Start_entity have|dobj|* *|*1|end_entity




In [101]:
cause_docs = [nlp(c["Sentence"]) for c in causal_sents]
#for doc, meta in nlp.pipe(messages, as_tuples=True):
    # do something with the docs and meta, e.g. add it to a custom attribute
  #  doc._.meta_data = meta

In [149]:
cause_docs[12:16]

[Stemming from these experiences , students tend to change their perspectives on their role as physicians and define the type of physician they wish to be .,
 To explain why n-3 PUFA decreases the level of Hcy , we examined the effect of n-3 PUFA on the mRNA expression of genes encoding the critical enzymes involved in methionine metabolism .,
 Our findings suggest that a low dose of THC produces subjective stress-relieving effects in line with those commonly reported among cannabis users , but that higher doses may non-specifically increase negative mood .,
 Prevention of diarrhea , severe respiratory tract infections , and anemia are likely to decrease the burden of wasting .]

In [103]:
negex_docs = [doc for doc in cause_docs if any([e._.negex for e in doc.ents])]


151

In [147]:
doc = nlp("I did not have sexual intercourse with that woman")
show_negex_entities([doc])

[{'I did not have sexual intercourse with that woman': [sexual intercourse,
   woman]}]

In [150]:
show_negex_entities(cause_docs[:10])

[{'It reported that higher facial symmetry was significantly associated with faster and less variable CRT in men , but not in women , and did not correlate with SRT or SRT variability .': [women,
   SRT,
   SRT,
   variability]},
 {'Although multivessel coronary artery disease is not associated with impaired reperfusion success assessed by cardiac magnetic resonance imaging , patients with multivessel disease are at a high risk of adverse clinical outcomes .': [associated with,
   impaired,
   reperfusion,
   success,
   assessed,
   cardiac magnetic resonance imaging,
   patients,
   multivessel disease,
   high risk,
   adverse clinical]}]

In [154]:
negex_docs[:3][1], len(negex_docs), len(cause_docs)

(Although multivessel coronary artery disease is not associated with impaired reperfusion success assessed by cardiac magnetic resonance imaging , patients with multivessel disease are at a high risk of adverse clinical outcomes .,
 151,
 1113)

In [222]:
from scify.networks import get_sdp_path

In [223]:
visualise_doc(doc)

In [181]:
[(tok.head, * tok.ancestors, tok.is_ancestor(tok), tok, tok.dep_) for tok in get_sdp_path(doc, 0, 7)]

[(have, have, False, I, 'nsubj'),
 (have, False, have, 'ROOT'),
 (have, have, False, sexual intercourse, 'dobj'),
 (sexual intercourse, sexual intercourse, have, False, with, 'prep'),
 (with, with, sexual intercourse, have, False, woman, 'pobj')]

In [240]:
doc[4].ancestors

In [171]:
show_tabs(doc)

token               lemma               POS    Tag    DEP    shape      is_alpha    is_stop
------------------  ------------------  -----  -----  -----  ---------  ----------  ---------
I                   -PRON-              PRON   PRP    nsubj  X          True        True
did                 do                  AUX    VBD    aux    xxx        True        True
not                 not                 PART   RB     neg    xxx        True        True
have                have                AUX    VB     ROOT   xxxx       True        True
sexual intercourse  sexual intercourse  NOUN   NN     dobj   xxxx xxxx  False       False
with                with                ADP    IN     prep   xxxx       True        True
that                that                DET    DT     det    xxxx       True        True
woman               woman               NOUN   NN     pobj   xxxx       True        False


'Printed Table above'

In [92]:
"""
NegEx Patterns
psuedo_negations - phrases that are false triggers, ambiguous negations, or double negatives
preceding_negations - negation phrases that precede an entity
following_negations - negation phrases that follow an entity
termination - phrases that cut a sentence in parts, for purposes of negation detection (.e.g., "but")
"""
#negex.get_patterns()

{'pseudo_patterns': [no further,
  not able to be,
  not certain if,
  not certain whether,
  not necessarily,
  without any further,
  without difficulty,
  without further,
  might not,
  not only,
  no increase,
  no significant change,
  no change,
  no definite change,
  not extend,
  not cause,
  not certain if,
  not certain whether,
  gram negative,
  not rule out,
  not ruled out,
  not been ruled out,
  not drain,
  no suspicious change,
  no interval change,
  no significant interval change],
 'preceding_patterns': [absence of,
  declined,
  denied,
  denies,
  denying,
  no sign of,
  no signs of,
  not,
  not demonstrate,
  symptoms atypical,
  doubt,
  negative for,
  no,
  versus,
  without,
  doesn't,
  doesnt,
  don't,
  dont,
  didn't,
  didnt,
  wasn't,
  wasnt,
  weren't,
  werent,
  isn't,
  isnt,
  aren't,
  arent,
  cannot,
  can't,
  cant,
  couldn't,
  couldnt,
  never,
  patient was not,
  without indication of,
  without sign of,
  without signs of,
  without